# Age of Empires II Voobly Matches Data Exploration

## Load libraries and set options/environment

In [2]:
# System
import sys
import os
import datetime
from datetime import date, timedelta
sys.path.append('C:/Users/Parikshit_verma/Documents/GitHub/AgeofEmpires2_Voobly/src/functions')

# Custom functions
from setup_sql_conn import setup_sql_conn,fetch_sql_data
from sql_read import sql_read

# Data Wrangling
import pandas as pd
import numpy as np
import re
from functools import reduce
from pandas_profiling import ProfileReport

# Visualization
import seaborn as sb
import matplotlib.pyplot as plt
import plotly

# Set options
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

## Read Data

In [84]:
db_conn,db_cursor,engine = setup_sql_conn('localhost','AOE2_VOOBLY')
match_data = sql_read('C:/Users/Parikshit_verma/Documents/GitHub/AgeofEmpires2_Voobly/Input/match_data_query.sql',db_conn)

Time taken to read from server : 4.1972975730896


## Data Mining & Profiling

In [85]:
# Match data cleanup
# Civilization dictionary
civ_map = ['Aztecs','Berbers','Britons','Bulgarians','Burmese','Byzantines','Celts','Chinese','Cumans','Ethiopians','Franks','Goths','Huns','Incas','Indians'
            ,'Italians','Japanese','Khmer','Koreans','Lithuanians','Magyars','Malay','Malians','Mayans','Mongols','Persians','Portuguese','Saracens','Slavs'
            ,'Spanish','Tatars','Teutons','Turks','Vietnamese','Vikings']
civ_map = {i.upper()[0:3] : i for i in civ_map}

match_data = (match_data
              .assign(Player_Rating = match_data['New_Rating'] - match_data['Points'],
                      Civilization = match_data['Civilization'].map(civ_map),
                      Match_date = pd.to_datetime(match_data['Match_date']),
                      Match_time = pd.to_datetime(match_data['Match_time'], format='%H:%M:%S').dt.hour)
              .drop(['Match_rating','Match_player_no','New_Rating','ladder',
                     'Trade_Profit','Tribute_Received','Tribute_Sent',
                     'Team'],axis=1)
              .dropna(subset=['Civilization']))

In [87]:
# Create differenced scores and stats
match_data_max = (match_data
                  .groupby(['Match_ID'])['Military_Score','Economy_Score','Technology_Score','Society_Score','Total_Score',
                                         'Units_Killed','Units_Lost','Buildings_Razed','Buildings_Lost','Units_Converted',
                                         'Food','Wood','Stone','Gold',
                                         'Map_Explored','Research_Count','Research_Percent',
                                         'Total_Wonders','Total_Castles','Relics_Captured','Relic_Gold','Villager_High',
                                         'Player_Rating'].max()
                  .add_suffix('_max')).reset_index()

match_data_min = (match_data
                  .groupby(['Match_ID'])['Military_Score','Economy_Score','Technology_Score','Society_Score','Total_Score',
                                         'Units_Killed','Units_Lost','Buildings_Razed','Buildings_Lost','Units_Converted',
                                         'Food','Wood','Stone','Gold',
                                         'Map_Explored','Research_Count','Research_Percent',
                                         'Total_Wonders','Total_Castles','Relics_Captured','Relic_Gold','Villager_High',
                                         'Player_Rating'].min()
                  .add_suffix('_min')).reset_index()

match_data = (match_data
              .merge(match_data_max,on = 'Match_ID')
              .merge(match_data_min,on = 'Match_ID'))

In [92]:
match_data['Military_Score'].apply(lambda x: (match_data['Military_Score'] - match_data['Military_Score_min']) if x ==  else (match_data['Military_Score'] - match_data['Military_Score_max']))

SyntaxError: invalid syntax (<ipython-input-92-3f55e21f16f0>, line 1)